<a href="https://colab.research.google.com/github/anmuhina/VkCloud/blob/master/lab1_regresssion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [43]:
df = pd.read_csv('AirQualityUCI.csv', delimiter=';')

In [44]:
#Приводим в нужному формату и числовому типу
df['CO(GT)'] = df['CO(GT)'].str.replace(',', '.')
df['CO(GT)'] = pd.to_numeric(df['CO(GT)'], errors='coerce')
df['T'] = df['T'].str.replace(',', '.')
df['T'] = pd.to_numeric(df['T'], errors='coerce')
df['C6H6(GT)'] = df['C6H6(GT)'].str.replace(',', '.')
df['C6H6(GT)'] = pd.to_numeric(df['C6H6(GT)'], errors='coerce')
df['RH'] = df['RH'].str.replace(',', '.')
df['RH'] = pd.to_numeric(df['RH'], errors='coerce')
df['AH'] = df['AH'].str.replace(',', '.')
df['AH'] = pd.to_numeric(df['AH'], errors='coerce')

In [45]:
#Обрабатываем пустые значения
df.replace(-200, pd.NA, inplace=True)
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
df['Date'].ffill(inplace=True)
df['Time'].ffill(inplace=True)
df['CO(GT)'].fillna(df['CO(GT)'].mean(), inplace=True)
df['PT08.S1(CO)'].fillna(df['PT08.S1(CO)'].mean(), inplace=True)
df['NMHC(GT)'].fillna(df['NMHC(GT)'].mean(), inplace=True)
df['C6H6(GT)'].fillna(df['C6H6(GT)'].mean(), inplace=True)
df['PT08.S2(NMHC)'].fillna(df['PT08.S2(NMHC)'].mean(), inplace=True)
df['NOx(GT)'].fillna(df['NOx(GT)'].mean(), inplace=True)
df['PT08.S3(NOx)'].fillna(df['PT08.S3(NOx)'].mean(), inplace=True)
df['NO2(GT)'].fillna(df['NO2(GT)'].mean(), inplace=True)
df['PT08.S4(NO2)'].fillna(df['PT08.S4(NO2)'].mean(), inplace=True)
df['PT08.S5(O3)'].fillna(df['PT08.S5(O3)'].mean(), inplace=True)
df['T'].fillna(df['T'].mean(), inplace=True)
df['RH'].fillna(df['RH'].mean(), inplace=True)
df['AH'].fillna(df['AH'].mean(), inplace=True)
df

<ipython-input-45-6d454c4c48ae>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Date'].ffill(inplace=True)
<ipython-input-45-6d454c4c48ae>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col:

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,2.60000,1360.000000,150.000000,11.900000,1046.000000,166.000000,1056.000000,113.000000,1692.000000,1268.000000,13.600000,48.900000,0.75780
1,10/03/2004,19.00.00,2.00000,1292.000000,112.000000,9.400000,955.000000,103.000000,1174.000000,92.000000,1559.000000,972.000000,13.300000,47.700000,0.72550
2,10/03/2004,20.00.00,2.20000,1402.000000,88.000000,9.000000,939.000000,131.000000,1140.000000,114.000000,1555.000000,1074.000000,11.900000,54.000000,0.75020
3,10/03/2004,21.00.00,2.20000,1376.000000,80.000000,9.200000,948.000000,172.000000,1092.000000,122.000000,1584.000000,1203.000000,11.000000,60.000000,0.78670
4,10/03/2004,22.00.00,1.60000,1272.000000,51.000000,6.500000,836.000000,131.000000,1205.000000,116.000000,1490.000000,1110.000000,11.200000,59.600000,0.78880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,04/04/2005,14.00.00,2.15275,1099.833166,218.811816,10.083105,939.153376,246.896735,835.493605,113.091251,1456.264598,1022.906128,18.317829,49.234201,1.02553
9467,04/04/2005,14.00.00,2.15275,1099.833166,218.811816,10.083105,939.153376,246.896735,835.493605,113.091251,1456.264598,1022.906128,18.317829,49.234201,1.02553
9468,04/04/2005,14.00.00,2.15275,1099.833166,218.811816,10.083105,939.153376,246.896735,835.493605,113.091251,1456.264598,1022.906128,18.317829,49.234201,1.02553
9469,04/04/2005,14.00.00,2.15275,1099.833166,218.811816,10.083105,939.153376,246.896735,835.493605,113.091251,1456.264598,1022.906128,18.317829,49.234201,1.02553


In [51]:
'''1. Базовая линейная регрессия с одним признаком. Предсказать уровень концентрации
оксида углерода (CO(GT)) на основе уровня концентрации бензола (C6H6(GT)) используя
простую линейную регрессию. Использовать метрики: R-квадрат, MSE'''

X = df[['C6H6(GT)']]
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Базовая линейная регрессия:")
print(f"R-квадрат: {r2:.2f}")
print(f"MSE: {mse:.2f}")

'''predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred})
print(predictions_df.head())'''

Базовая линейная регрессия:
R-квадрат: 0.68
MSE: 0.57


"predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred})\nprint(predictions_df.head())"

**Таким образом, модель является не очень хорошей, поскольку значение MSE очень велико (должно быть как можно меньше), а значение R-квадрат, наоборот, должно быть как можно ближе к 1. То есть на основании одного признака C6H6(GT) сложно верно предсказать концентрацию CO(GT) с помощью данной модели.**

In [32]:
'''2. Множественная линейная регрессия. Предсказать уровень концентрации оксида
углерода (CO(GT)) на основе нескольких признаков (например, C6H6(GT), T, RH, NO2(GT)).
Использовать метрики: R-квадрат, MSE'''

X = df[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = LinearRegression()
model_2.fit(X_train, y_train)
y_pred_2 = model_2.predict(X_test)

r2_2 = r2_score(y_test, y_pred_2)
mse_2 = mean_squared_error(y_test, y_pred_2)

print("Множественная линейная регрессия:")
print(f"R-квадрат: {r2_2:.2f}")
print(f"MSE: {mse_2:.2f}")

'''predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred_2})
print(predictions_df.head())'''

Множественная линейная регрессия:
R-квадрат: 0.74
MSE: 0.46


**Данная модель лучше предыдущей, поскольку за счет увеличения количества признаков удалость добиться увеличения R-квадрат и уменьшения MSE, к чему мы и стремимся.**

In [33]:
'''3. Стандартизация признаков и линейная регрессия. Повторить задачу 2, но перед
обучением модели стандартизировать признаки, используя StandardScaler. Использовать
метрики: R-квадрат, MSE'''

X = df[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) #Находит среднее и стандартное отклонение для признаков в тренировочной выборке и преобразует их, чтобы они имели среднее 0 и стандартное отклонение 1
X_test_scaled = scaler.transform(X_test) #Применяет те же параметры, найденные на тренировочной выборке, к тестовой выборке

model_3 = LinearRegression()
model_3.fit(X_train_scaled, y_train)
y_pred_3 = model_3.predict(X_test_scaled)

r2_3 = r2_score(y_test, y_pred_3)
mse_3 = mean_squared_error(y_test, y_pred_3)

print("Стандартизация и линейная регрессия:")
print(f"R-квадрат: {r2_3:.2f}")
print(f"MSE: {mse_3:.2f}")

'''predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred_3})
print(predictions_df.head())'''

Стандартизация и линейная регрессия:
R-квадрат: 0.74
MSE: 0.46


**Получены те же самые значения.**

In [46]:
'''4. Линейная регрессия с L1 регуляризацией (Lasso). Предсказать уровень концентрации
оксида углерода (CO(GT)) на основе всех доступных признаков, используя линейную
регрессию с L1 регуляризацией (Lasso). Подобрать оптимальный параметр регуляризации
(alpha) с помощью кросс-валидации. Использовать метрики: R-квадрат, MSE, также
оценить количество ненулевых коэффициентов'''

#Превращаем дату в число
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df = df.drop(['Date'], axis=1)

#Превращаем время в число
def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split('.'))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds
df['Time'] = df['Time'].apply(time_to_seconds)

print(df)

X = df.drop(['CO(GT)'], axis=1)
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso_cv = LassoCV(cv=5)
lasso_cv.fit(X_train_scaled, y_train)
model_4 = Lasso(alpha=lasso_cv.alpha_)
model_4.fit(X_train_scaled, y_train)
y_pred_4 = model_4.predict(X_test_scaled)

r2_4 = r2_score(y_test, y_pred_4)
mse_4 = mean_squared_error(y_test, y_pred_4)
non_zero_coeffs = np.sum(model_4.coef_ != 0)

print("\nРезультаты Lasso регрессии:")
print(f"R-квадрат: {r2_4:.2f}")
print(f"MSE: {mse_4:.2f}")
print(f"Оптимальный alpha: {lasso_cv.alpha_:.4f}")
print(f"Ненулевые коэффициенты: {non_zero_coeffs}")

'''predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred_4})
print(predictions_df.head())'''

       Time   CO(GT)  PT08.S1(CO)    NMHC(GT)   C6H6(GT)  PT08.S2(NMHC)  \
0     64800  2.60000  1360.000000  150.000000  11.900000    1046.000000   
1     68400  2.00000  1292.000000  112.000000   9.400000     955.000000   
2     72000  2.20000  1402.000000   88.000000   9.000000     939.000000   
3     75600  2.20000  1376.000000   80.000000   9.200000     948.000000   
4     79200  1.60000  1272.000000   51.000000   6.500000     836.000000   
...     ...      ...          ...         ...        ...            ...   
9466  50400  2.15275  1099.833166  218.811816  10.083105     939.153376   
9467  50400  2.15275  1099.833166  218.811816  10.083105     939.153376   
9468  50400  2.15275  1099.833166  218.811816  10.083105     939.153376   
9469  50400  2.15275  1099.833166  218.811816  10.083105     939.153376   
9470  50400  2.15275  1099.833166  218.811816  10.083105     939.153376   

         NOx(GT)  PT08.S3(NOx)     NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
0     166.000000   1056.00

**Таким образом, Лассо-регуляризация помогла еще немного увеличить значение R-квадрат и уменьшить значение MSE, что позволило нам получить еще более удачную модель. Также получили оптимальное значение alpha и число ненулевых коэффициетов, то есть число параметров, используемых в модели**

In [47]:
'''5. Линейная регрессия с L2 регуляризацией (Ridge). Предсказать уровень концентрации
оксида углерода (CO(GT)) на основе всех доступных признаков, используя линейную
регрессию с L2 регуляризацией (Ridge). Подобрать оптимальный параметр регуляризации
(alpha) с помощью кросс-валидации. Использовать метрики: R-квадрат, MSE'''

X = df.drop(['CO(GT)'], axis=1)
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge_cv = RidgeCV(cv=5)
ridge_cv.fit(X_train_scaled, y_train)

model_5 = Ridge(alpha=ridge_cv.alpha_)
model_5.fit(X_train_scaled, y_train)
y_pred_5 = model_5.predict(X_test_scaled)

r2_5 = r2_score(y_test, y_pred_5)
mse_5 = mean_squared_error(y_test, y_pred_5)

print("Ridge регрессия:")
print(f"R-квадрат: {r2_5:.2f}")
print(f"MSE: {mse_5:.2f}")
print(f"Оптимальный alpha: {ridge_cv.alpha_:.4f}")

'''predictions_df = pd.DataFrame({'Фактическое значение': y_test, 'Предсказанное значение': y_pred_5})
print(predictions_df.head())'''

Ridge регрессия:
R-квадрат: 0.82
MSE: 0.33
Оптимальный alpha: 0.1000


**Можем заметить, что данная регуляризация дала точно такой же результат, как и в предыдущем случае (изменилось только значение alpha), то есть улучшения добиться не удалось.**

In [48]:
'''6. Кросс-валидация для оценки модели. Использовать K-fold кросс-валидацию
(например, с K=5 или K=10) для оценки производительности модели множественной
линейной регрессии (пункт 2). Использовать метрики: R-квадрат, MSE по всем фолдам'''

X = df[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]
y = df['CO(GT)']

model_6 = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = cross_val_score(model_6, X, y, cv=kf, scoring='r2')
mse_scores = cross_val_score(model_6, X, y, cv=kf, scoring='neg_mean_squared_error')

print("Кросс-валидация:")
print(f"Средний R-квадрат: {r2_scores.mean():.2f}")
print(f"Средний MSE: {-mse_scores.mean():.2f}")

Кросс-валидация:
Средний R-квадрат: 0.73
Средний MSE: 0.46


In [49]:
'''7. Регуляризация с кросс-валидацией для выбора параметра alpha (Lasso или Ridge).
Использовать кросс-валидацию для выбора оптимального значения параметра
регуляризации alpha для Lasso или Ridge регрессии. Например, использовать RidgeCV или
LassoCV из scikit-learn. Использовать метрики: R-квадрат, MSE, оптимальное значение
alpha(??)'''

X = df.drop(columns=['CO(GT)'])
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
alphas = [0.1, 1.0, 10.0, 100.0]

#Ridge
ridge_model = RidgeCV(alphas=alphas)
ridge_model.fit(X_train, y_train)
ridge_predictions = ridge_model.predict(X_test)
ridge_r2 = r2_score(y_test, ridge_predictions)
ridge_mse = mean_squared_error(y_test, ridge_predictions)
ridge_optimal_alpha = ridge_model.alpha_

'''#Lasso
lasso_model = LassoCV(alphas=alphas, cv=5)
lasso_model.fit(X_train, y_train)
lasso_predictions = lasso_model.predict(X_test)
lasso_r2 = r2_score(y_test, lasso_predictions)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
lasso_optimal_alpha = lasso_model.alpha_
non_zero_coefficients = (lasso_model.coef_ != 0).sum()'''

print("Результаты для Ridge регрессии:")
print(f"Оптимальное значение alpha: {ridge_optimal_alpha:.4f}")
print(f"R-квадрат: {ridge_r2:.2f}")
print(f"MSE: {ridge_mse:.2f}")

'''print("\nРезультаты для Lasso регрессии:")
print(f"Оптимальное значение alpha: {lasso_optimal_alpha:.4f}")
print(f"R-квадрат: {lasso_r2:.2f}")
print(f"MSE: {lasso_mse:.2f}")
print(f"Количество ненулевых коэффициентов: {non_zero_coefficients}")'''

Результаты для Ridge регрессии:
Оптимальное значение alpha: 10.0000
R-квадрат: 0.82
MSE: 0.33


'print("\nРезультаты для Lasso регрессии:")\nprint(f"Оптимальное значение alpha: {lasso_optimal_alpha:.4f}")\nprint(f"R-квадрат: {lasso_r2:.2f}")\nprint(f"MSE: {lasso_mse:.2f}")\nprint(f"Количество ненулевых коэффициентов: {non_zero_coefficients}")'

In [52]:
'''8. Полиномиальная регрессия. Создать полиномиальные признаки (например, до степени
2 или 3) из исходных признаков и использовать их для обучения линейной регрессии.
Сравнить производительность с обычной линейной регрессией. Использовать метрики: R-
квадрат, MSE'''

X = df[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]
y = df['CO(GT)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

y_pred_poly = model_poly.predict(X_test_poly)

r2_poly = r2_score(y_test, y_pred_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)

print("Полиномиальная регрессия:")
print(f"R-квадрат: {r2_poly:.2f}")
print(f"MSE: {mse_poly:.2f}")

print("\nСравнение с обычной линейной регрессией (из задачи 2):")
print(f"R-квадрат: {r2_2:.2f}")
print(f"MSE: {mse_2:.2f}")

Полиномиальная регрессия:
R-квадрат: 0.77
MSE: 0.41

Сравнение с обычной линейной регрессией (из задачи 2):
R-квадрат: 0.74
MSE: 0.46


**Таким образом, полиномиальная регрессия третьей степени позволила получить немного более хороший результат в сравнении с обычной множественной линейной регрессией (задача 2). Однако эти результаты хуже, чем при использовании регуляризации и кросс-валидации (задачи 4-7), что может быть вызвано, например, тем, что выбранная степень полинома не является оптимальной или же тем, что между признаками отсутствует сильная нелинейная зависимость.**

In [53]:
'''9. Сравнение моделей и интерпретация результатов. Сравнить производительность всех
построенных моделей (простая линейная регрессия, множественная линейная регрессия,
Lasso, Ridge, полиномиальная регрессия). Проанализировать коэффициенты моделей и
сделать выводы о важности различных признаков для прогнозирования CO(GT)'''

print("Сравнение моделей:")
print("Модель 1 (базовая): R-квадрат =", r2, ", MSE =", mse)
print("Модель 2 (множественная): R-квадрат =", r2_2, ", MSE =", mse_2)
print("Модель 3 (стандартизация): R-квадрат =", r2_3, ", MSE =", mse_3)
print("Модель 4 (Lasso): R-квадрат =", r2_4, ", MSE =", mse_4)
print("Модель 5 (Ridge): R-квадрат =", r2_5, ", MSE =", mse_5)
print("Модель 8 (полиномиальная): R-квадрат =", r2_poly, ", MSE =", mse_poly)

print("\nИнтерпретация:")
print("Коэффициенты модели 1:", model.coef_)
print("Коэффициенты модели 2:", model_2.coef_)
print("Коэффициенты модели 3:", model_3.coef_)
print("Коэффициенты модели 4):", model_4.coef_)
print("Коэффициенты модели 5:", model_5.coef_)
print("Коэффициенты модели 8:", model_poly.coef_)

Сравнение моделей:
Модель 1 (базовая): R-квадрат = 0.6759706545001338 , MSE = 0.5726323698065474
Модель 2 (множественная): R-квадрат = 0.7399709276742759 , MSE = 0.4595295641349219
Модель 3 (стандартизация): R-квадрат = 0.7399709276742759 , MSE = 0.4595295641349219
Модель 4 (Lasso): R-квадрат = 0.8160044464656974 , MSE = 0.3251613204713852
Модель 5 (Ridge): R-квадрат = 0.8159784664063148 , MSE = 0.3252072330505346
Модель 8 (полиномиальная): R-квадрат = 0.7669342276221685 , MSE = 0.41187937886189446

Интерпретация:
Коэффициенты модели 1: [0.14626042]
Коэффициенты модели 2: [ 0.11809261 -0.0005278   0.00801561  0.00919984]
Коэффициенты модели 3: [ 0.85167588 -0.00452418  0.13495957  0.40172757]
Коэффициенты модели 4): [ 0.09385304  0.1613549   0.12780646  0.60118084 -0.47062299  0.62995644
  0.02153251  0.13254402  0.45155407 -0.06691445 -0.10847483 -0.05803643
 -0.10521353 -0.08203997  0.01086724  0.02498184]
Коэффициенты модели 5: [ 0.09810158  0.17539114  0.12618432  0.67933436 -0.612

На основании полученных результатов можно провести анализ производительности различных моделей и интерпретировать их коэффициенты для оценки важности признаков в предсказании уровня концентрации оксида углерода (CO(GT)).

Заметим, что модели с Lasso и Ridge регуляризацией оказались наилучшими, обеспечивая наивысшее значение R-квадрат и наименьшее значение MSE. Это говорит о том, что регуляризация помогает улучшить модель, предотвращая переобучение и лучше справляясь с многоколлинеарностью.

Простая линейная регрессия показала худшие результаты по сравнению с множественной регрессией и моделями с регуляризацией, что указывает на важность многомерного анализа.

В данном случае нет сильной нелинейной зависимости между признаками и целевой переменной, так как полиномиальная регрессия показала результаты хуже, чем линейные модели с регуляризацией.

Интерпретация коэффициентов (анализ коэффициентов позволяет выявить важность отдельных признаков, что может дать представление о процессе, стоящем за изменениями уровня CO(GT)):
- если признаков с близкими к нулю коэффициентами много, это может указывать на их низкую значимость в модели
- положительные значения указывают на прямую зависимость, а отрицательные — на обратную
- коэффициенты Lasso и Ridge не только показывают влияние признаков, но и могут указывать на признаки, которые были обнулены в Lasso (т.е. не учитываются в модели), тем самым позволяя выделить наиболее значимые признаки